In [32]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import math
# from tqdm import tqdm_notebook as tqdm

In [2]:
batch_path = '/Users/ivan_zorin/Documents/AIRI/data/batt/'

# pkl_file = os.path.join(batch_path, 'batt_data.pkl')
# if os.path.exists(pkl_file):
#     with open(pkl_file, 'rb') as f:
#         bat_dict = pickle.load(f)

In [3]:
batch1 = pickle.load(open(batch_path+'batch1.pkl', 'rb'))
#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

In [4]:
numBat1 = len(batch1.keys())
numBat1

42

In [5]:
batch2 = pickle.load(open(batch_path+'batch2.pkl','rb'))

In [6]:
# There are four cells from batch1 that carried into batch2, we'll remove the data from batch2
# and put it with the correct cell from batch1
batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16']
batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
add_len = [662, 981, 1060, 208, 482];

In [7]:
for i, bk in enumerate(batch1_keys):
    batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
    for j in batch1[bk]['summary'].keys():
        if j == 'cycle':
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
        else:
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
    last_cycle = len(batch1[bk]['cycles'].keys())
    for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
        batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]

In [8]:
del batch2['b2c7']
del batch2['b2c8']
del batch2['b2c9']
del batch2['b2c15']
del batch2['b2c16']

In [9]:
numBat2 = len(batch2.keys())
numBat2

42

In [10]:
batch3 = pickle.load(open(batch_path+'batch3.pkl','rb'))
# remove noisy channels from batch3
del batch3['b3c37']
del batch3['b3c2']
del batch3['b3c23']
del batch3['b3c32']
del batch3['b3c42']
del batch3['b3c43']

In [11]:
numBat3 = len(batch3.keys())
numBat3

41

In [12]:
numBat = numBat1 + numBat2 + numBat3
numBat

125

In [13]:
bat_dict = {**batch1, **batch2, **batch3}

In [14]:
bat_dict['b1c0'].keys()

dict_keys(['cycle_life', 'charge_policy', 'summary', 'cycles'])

In [15]:
bat_dict['b1c0']['cycles']['0'].keys()

dict_keys(['I', 'Qc', 'Qd', 'Qdlin', 'T', 'Tdlin', 'V', 'dQdV', 't'])

In [33]:
math.isnan(bat_dict['b3c22']['cycle_life'].item())

True

In [17]:
labels = {bat_id: bat_dict[bat_id]['cycle_life'].item() for bat_id in bat_dict.keys()}

In [34]:
labels_nan = [value for value in labels.values() if math.isnan(value)]

In [36]:
len(labels_nan)

21

In [ ]:
for i in bat_dict.keys():
    plt.plot(bat_dict[i]['summary']['cycle'], bat_dict[i]['summary']['QD'])
plt.xlabel('Cycle Number')
plt.ylabel('Discharge Capacity (Ah)')

In [ ]:
pkl_file = os.path.join(batch_path, 'batt_data.pkl')
if not os.path.exists(pkl_file):
    with open(pkl_file, 'wb') as f:
        pickle.dump(bat_dict, f)

In [ ]:
batts = list(bat_dict.keys())

In [ ]:
bat_dict[batts[0]]

### Train and Test Split
If you are interested in using the same train/test split as the paper, use the indices specified below

In [ ]:
test_ind = np.hstack((np.arange(0,(numBat1+numBat2),2),83))
train_ind = np.arange(1,(numBat1+numBat2-1),2)
secondary_test_ind = np.arange(numBat-numBat3,numBat);